В этом ноутбуке я поазываю пример работы с библиотекой UDPipe для морфологического и синтаксического анализа текста.

In [ ]:
# Команда !pip устанавливает (install) библиотеки для Питона.
# Если запустить !pip freeze, можно увидеть, какие библиотеки у вас уже есть.
# Сначала установим сам udpipe.
!pip install ufal.udpipe
# И еще установим дополнительную библиотеку.
!pip install corpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 936.8/936.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 425.5/425.5 kB 8.0 MB/s eta 0:00:00


In [ ]:
# Что это за библиотека - UDPipe? Это бибилиотека для "tagging, lemmatization
# and syntactic analysis of CoNLL-U input" - теггирования, лемматизации и
# синтаксического анализа данных в формате CoNLL-U. Следовательно,
# можно взять файлы с размеченными текстами с Universal Dependencies
# для любого языка и обучить модель для аннотирования текста.
# https://github.com/ufal/udpipe
# Теперь скачаем предобученную модель udpipe для русского языка,
# созданную командой rusvectores https://rusvectores.org/ru/ .
# Библиотека udpipe позволяет обучать свои модели для разных языков
!wget 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
modelfile = '/content/udpipe_syntagrus.model'

--2024-05-15 05:55:25--  https://rusvectores.org/static/models/udpipe_syntagrus.model
Resolving rusvectores.org (rusvectores.org)... 172.104.228.108
Connecting to rusvectores.org (rusvectores.org)|172.104.228.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40616122 (39M)
Saving to: ‘udpipe_syntagrus.model’

udpipe_syntagrus.mo 100%[===================>]  38.73M  13.3MB/s    in 2.9s    

2024-05-15 05:55:29 (13.3 MB/s) - ‘udpipe_syntagrus.model’ saved [40616122/40616122]



In [ ]:
# Импортируем инструмент для загрузки модели.
from corpy.udpipe import Model

In [ ]:
# Распаковываем модель. Она у нас сейчас на виртуальном диске в папке content.
m = Model('/content/udpipe_syntagrus.model')

In [ ]:
# Основной метод для разметки текста на основе модели - это process.
parsed = m.process('Тюмень - лучший город Земли!')

In [ ]:
# Частая проблема некоммерческих бибилиотек в том, что они не user-friendly.
# Чтоы добраться до результата парсинга, придется немного попотеть.
# Если мы сейчас просто попробуем распечатать результат, то будет пшик.
print(parsed)
# generator означает, что у нас есть набор каких-то элементов внутри этой
# переменной, но print их просто так не напечатает.
for p in parsed:
    print(p)
# Более того, стоит нам один раз пройтись по этому объекту, он становится пустым.
# Поэтому создадим снова объект parsed и созраним его в список - от греха подальше.
parsed = list(m.process('Тюмень - лучший город Земли!'))

<generator object Model.process at 0x780f29700e40>
<Swig Object of type 'sentence *' at 0x780f2971f4f0>


In [ ]:
# Вот теперь у нас сохранено одно распарщенное предложение, но мы не можем
# добраться до его содержания.
print(parsed)

[<Swig Object of type 'sentence *' at 0x780f2971e9b0>]


In [ ]:
# Используем функцию pprint.
from corpy.udpipe import pprint

In [ ]:
# Вот теперь мы видим, что UDPipe умеет парсить.
# Смотрите, там есть и морфология, и синтаксис. И все размечено правильно.
pprint(list(parsed))

[Sentence(
   comments=[
     '# newdoc',
     '# newpar',
     '# sent_id = 1',
     '# text = Тюмень - лучший город Земли!'],
   words=[
     Word(id=0, <root>),
     Word(id=1,
          form='Тюмень',
          lemma='тюмень',
          upostag='PROPN',
          feats='Animacy=Inan|Case=Nom|Gender=Fem|Number=Sing',
          head=4,
          deprel='nsubj'),
     Word(id=2, form='-', lemma='-', upostag='PUNCT', head=1, deprel='punct'),
     Word(id=3,
          form='лучший',
          lemma='лучший',
          upostag='ADJ',
          feats='Case=Nom|Degree=Pos|Gender=Masc|Number=Sing',
          head=4,
          deprel='amod'),
     Word(id=4,
          form='город',
          lemma='город',
          upostag='NOUN',
          feats='Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing',
          head=0,
          deprel='root'),
     Word(id=5,
          form='Земли',
          lemma='земля',
          upostag='PROPN',
          feats='Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing',

In [ ]:
# UDPipe нашел одно предложение в тексте.
for p in parsed:
    print(p.comments[2])
# Такие сложности в доступом отчасти связаны с тем, что инструмент
# изначально написан на C++.

# sent_id = 1


In [ ]:
# Доберемся до начальных форм слов при помощи цикла for.
for w in parsed[0].words:
    print(w.lemma)
# Можем сохранить их в список при помощи list comprehension.
[w.lemma for w in parsed[0].words]

<root>
тюмень
-
лучший
город
земля
!


['<root>', 'тюмень', '-', 'лучший', 'город', 'земля', '!']

In [ ]:
# А теперь вытащим все именованные сущности.
for w in parsed[0].words:
    if w.upostag == 'PROPN':
        print(w.form)

Тюмень
Земли


Повторю, UDPipe  - это инструмент для теггирования, лемматизации и синтаксического анализа. (В разметке выше синтаксис есть, кстати, в тэгах head и deprel. Про них будем говорить отдельно.) Для более сложных задач нужно достраивать "лингвистическую трубу" после обработки текста UDPipe'ом. Но я решила законичть этот ноутбук маленьким бонусом. В ботах часто возникает необходимость поставить слово в нужную форму после числительного. Для русского языка есть целый отдельный инструмент для такой задачи - pymorphy2.

In [ ]:
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 49.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=306b4bc91f8447e6157aaa6f07f036b28c81dfc575feee40b35b514e51c79a1e
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [ ]:
coin = morph.parse('монета')[0]

In [ ]:
print(f'ноль {coin.make_agree_with_number(0).word}')
print(f'одна {coin.make_agree_with_number(1).word}')
print(f'две {coin.make_agree_with_number(2).word}')
print(f'пять {coin.make_agree_with_number(5).word}')
print(f'двадцать одна {coin.make_agree_with_number(21).word}')
print(f'тысяча {coin.make_agree_with_number(1000).word}')
print(f'сто тысяч {coin.make_agree_with_number(100000).word}')
# А если захотите перевести число, написанное цифрой, в буквенный вид,
# то есть библиотека num2words.

ноль монет
одна монета
две монеты
пять монет
двадцать одна монета
тысяча монет
сто тысяч монет
